## Import modules

Set your user name as a case, pointing to your path to documents and tinyir.jar

In [ ]:
// set your case once
val (doc_dir: String, files_path: String) = System.getProperties().get("user.name").toString match {
    case "Yarden-"  => ("../documents", "../")
    case "Max"  => ("/Users/Max/Coding/ETH/Information_Retrieval_AS16/scala_practice/documents", "/Users/Max/Coding/ETH/Information_Retrieval_AS16/scala_practice/files/")
}

In [ ]:
classpath.addPath(files_path + "tinyir-1.1.jar")

In [ ]:
import scala.xml.XML
import ch.ethz.dal.tinyir._
import com.github.aztek.porterstemmer.PorterStemmer

In [ ]:
import scala.io.Source  // for importing txt files
import java.io._  // for saving txt files
// import scala.collection.mutable.HashMap  //HashMap used for counting elements in linear time

In [ ]:
// import scala.util.Random
import scala.collection.mutable.{Map => MutMap, HashMap => MutHashMap}
// enables "mutable lists"
// import scala.collection.mutable.ListBuffer  
import scala.collection.mutable.{Set => MutSet}

In [ ]:
val timeit = new util.StopWatch

## Define classes and functions

In [ ]:
def average[T]( ts: Iterable[T] )( implicit num: Numeric[T] ) = {
  num.toDouble( ts.sum ) / ts.size
}

In [ ]:
def token_filter(text_body: String) = {
    processing.StopWords.filterOutSW(
        processing.Tokenizer.tokenize(text_body.
                                      replaceAll("\\P{L}+", " "))
    ).
    map(x => PorterStemmer.stem(x)).filter(_.trim.nonEmpty).toList
}

In [ ]:
class xml_doc (file_path: String) {
    def get_doc(): xml.Elem = {
        XML.loadFile(file_path: String)
    }    
    
    def text() = {
        (get_doc() \\ "DOC" \\ "TEXT").text
    }
    
    def head() = {
        (get_doc() \\ "DOC" \\ "HEAD").text
    }

    def id() = {
        (get_doc() \\ "DOC" \\ "DOCNO").text.trim
    }
    
    def tokens() = {
        token_filter(head() ++ text())
    }
    
    def hash_tokens() = {
        tokens().map(x => x.hashCode())
    }
}

In [ ]:
def list_docs (path: String) = {  // : Array[java.io.File]
        new java.io.File(path).listFiles.map(x => x.toString())
    }
val numPattern = "[0-9]+".r

In [ ]:
val token_hash = MutHashMap[String, Int]() // token -> hash

def create_hash_doc_subset(star_count: Int, end_count: Int,
                           file_list: Array[String],
                           token_hash_map: MutHashMap[String, Int] = token_hash) = {
    val id_htoken = MutHashMap[Int, List[Int]]() // forward index, docID to tokens
    val htoken_id = MutHashMap[Int, List[Int]]()  // inverse index, tokens to docID
    val id_name = MutHashMap[Int, String]()  // inverse index, tokens to docID
    val name_id = MutHashMap[String, Int]()  // inverse index, tokens to docID
    var counter = star_count
    while (counter < end_count){
        var cur_doc = new xml_doc(file_list(counter))
        // get token from XML, then hash, or create hashes "on the fly"
        var cur_htoken = cur_doc.tokens.map(x => token_hash_map.getOrElseUpdate(x, token_hash_map.size))
        id_htoken += counter -> cur_htoken
        
        // update the inverse mapping, from (hashed) tokens to docID
        cur_htoken.distinct.foreach(
            (token: Int) => htoken_id(token) = htoken_id.getOrElseUpdate(token, List[Int]()) ++ List(counter)
        )
        
        id_name(counter) = cur_doc.id
        name_id(cur_doc.id) = counter
        
        counter += 1
        if (counter % 100 == 0) println(s"iteration $counter")
    }
    (id_htoken, htoken_id, token_hash_map, id_name, name_id)
}

In [ ]:
def write_int_to_intList(data: MutHashMap[Int, List[Int]], filename: String) = {

    val bw = new BufferedWriter(new FileWriter(new File(filename)))
    val iter = data.keys.iterator
    while(iter.hasNext){
        var elem = iter.next()
        var values = data(elem).toList
//         if(values.length>0){
            bw.write(elem+" "+values.mkString(" "))
            bw.newLine
//         }    
    }   
    bw.close()
}

def write_int_string(data: MutHashMap[Int, String], filename: String) = {

    val bw = new BufferedWriter(new FileWriter(new File(filename)))
    val iter = data.keys.iterator
    while(iter.hasNext){
        var elem = iter.next()
        var values = data(elem).toList
        if(values.length>0){
            bw.write(elem+" "+values.mkString(""))
            bw.newLine
        }    
    }   
    bw.close()
}

def write_string_int(data: MutHashMap[String, Int], filename: String) = {

    val bw = new BufferedWriter(new FileWriter(new File(filename)))
    val iter = data.keys.iterator
    while(iter.hasNext){
        var elem = iter.next()
        bw.write(elem+" "+data(elem).toString)
        bw.newLine
    }   
    bw.close()
}

def write_int_to_int(data: MutHashMap[Int, Int], filename: String) = {

    val bw = new BufferedWriter(new FileWriter(new File(filename)))
    val iter = data.keys.iterator
    while(iter.hasNext){
        var elem = iter.next()
        var value = data(elem)
            bw.write(elem+" "+value)
            bw.newLine
    }   
    bw.close()
}

In [ ]:
def load_mutmap_int_intList(path: String, mutmap: MutHashMap[Int, List[Int]]) = {
    val lines = Source.fromFile(path).getLines.toList
    for (line <- lines){
        val line_split = line.split(" ", -1).filter(_.trim.length > 0)
        mutmap(line_split.head.toInt) = 
            line_split.tail.map(x => x.toInt).toList
    }
}

def load_mutmap_int_string(path: String, mutmap: MutHashMap[Int, String]) = {
    val lines = Source.fromFile(path).getLines.toList
    for (line <- lines){
        val line_split = line.split(" ") // .filter(_.trim.length > 0)
        mutmap(line_split.head.toInt) = 
            line_split.last
    }
}

def load_mutmap_string_int(path: String, mutmap: MutHashMap[String, Int]) = {
    val lines = Source.fromFile(path).getLines.toList
    for (line <- lines){
//         val line_split = line.split(" ", -1)
        val line_split = line.split(" ") // .filter(_.trim.length > 0)
        mutmap(line_split.head) = 
            line_split.last.toInt
    }
}

def load_mutmap_int_int(path: String, mutmap: MutHashMap[Int, Int]) = {
    val lines = Source.fromFile(path).getLines.toList
    for (line <- lines){
        val line_split = line.split(" ", -1).filter(_.trim.length > 0)
        mutmap(line_split.head.toInt) = 
            line_split.last.toInt
    }
}

In [ ]:
val mb = 1024*1024
val runtime = Runtime.getRuntime
def print_memory() = {
    println(s"Used Memory:  " + (runtime.totalMemory - runtime.freeMemory) / mb)
    println(s"Free Memory:  " + runtime.freeMemory / mb)
    println(s"Total Memory: " + runtime.totalMemory / mb)
    println(s"Max Memory:   " + runtime.maxMemory / mb)
}

In [ ]:
val train_list = list_docs(doc_dir)

In [ ]:
val PATH_id_htoken = files_path + "id_htoken.txt"
val PATH_htoken_id = files_path + "htoken_id.txt"
val PATH_id_name = files_path + "id_name.txt"
val PATH_name_id = files_path + "name_id.txt"
val PATH_token_hash = files_path + "token_hash.txt"

val PATH_prun_htoken_collectfreq = files_path + "prun_htoken_collectfreq.txt"
val PATH_prun_htoken_id = files_path + "prun_htoken_id.txt"
val PATH_prun_id_htoken = files_path + "prun_id_htoken.txt"

# Importing data files and creating maps
# # not run

In [ ]:
// time it
timeit.start

val (id_htoken, htoken_id, token_hash, 
     id_name, name_id) = create_hash_doc_subset(0, 100000, train_list)

In [ ]:
// time it
timeit.uptonow / 60.0
// 87.56585954758334 , in minutes with 6GB
// 67.88821773650001 , in minutes with 7GB

In [ ]:
print_memory()

// Used Memory:  3981
// Free Memory:  1814
// Total Memory: 5796
// Max Memory:   5796

## Save to file

In [ ]:
write_int_to_intList(id_htoken, PATH_id_htoken)

In [ ]:
write_int_to_intList(htoken_id, PATH_htoken_id)

In [ ]:
write_int_string(id_name, PATH_id_name)

In [ ]:
write_string_int(name_id, PATH_name_id)

In [ ]:
write_string_int(token_hash, PATH_token_hash)

## Load from file

In [ ]:
// time it
timeit.start

val id_htoken: MutHashMap[Int, List[Int]] = MutHashMap[Int, List[Int]]()
val htoken_id: MutHashMap[Int, List[Int]] = MutHashMap[Int, List[Int]]()
val id_name: MutHashMap[Int, String] = MutHashMap[Int, String]()
val token_hash: MutHashMap[String, Int] = MutHashMap[String, Int]()
val name_id: MutHashMap[String, Int] = MutHashMap[String, Int]()

In [ ]:
load_mutmap_int_intList(PATH_id_htoken, id_htoken)
load_mutmap_int_intList(PATH_htoken_id, htoken_id)
load_mutmap_int_string(PATH_id_name, id_name)
load_mutmap_string_int(PATH_token_hash, token_hash)
load_mutmap_string_int(PATH_name_id, name_id)

// confirm load successful
// test_load_mutmap_id_htoken == id_htoken
// test_load_mutmap_htoken_id == htoken_id
// test_load_mutmap_id_name == id_name
// test_load_mutmap_token_hash == token_hash
// test_load_mutmap_name_id == name_id

In [ ]:
// time it
timeit.uptonow / 60.0
// 1.4485827969833334 , in minutes
// 1.3195113773833334 , in minutes

In [ ]:
print_memory()

// Used Memory:  3468
// Free Memory:  649
// Total Memory: 4117
// Max Memory:   5461

## Prune vocabulary, collection and document frequencies

In [ ]:
// htoken_id.mapValues(v => v.length).size
// 1356183
// htoken_id.mapValues(v => v.length).filter(_._2 > 5 - 1).size
// 176866
// reduction factor of ~7.67

val prun_threshold = 5
val pruned_token_set = htoken_id.mapValues(v => v.length).
    filter(_._2 > prun_threshold - 1).keys.toSet

In [ ]:
// time it
timeit.start

val prun_htoken_collectfreq: MutHashMap[Int, Int] = 
    MutHashMap(
        id_htoken.flatMap{ case (k,v) => v.filter(pruned_token_set.contains(_)) }.
        groupBy(identity).mapValues(_.size)
        .toSeq:_*)

prun_htoken_collectfreq.size

timeit.uptonow / 60.0
// 7.0919255263  , in minutes
// 0.39257662956666667 , in minutes

In [ ]:
// time it
timeit.start

val prun_htoken_id: MutHashMap[Int, List[Int]] = 
    MutHashMap(
        htoken_id.filterKeys(
            pruned_token_set.contains(_)
        ).toSeq:_*)

prun_htoken_id.size

timeit.uptonow / 60.0
// 0.012546176999999999 , in minutes

In [ ]:
// time it
timeit.start

val prun_id_htoken: MutHashMap[Int, List[Int]] = 
    MutHashMap(
//         id_htoken.flatMap{ case (k,v) => (k, v.filter(pruned_token_set.contains(_))) }.
        id_htoken.mapValues{ v => v.filter(pruned_token_set.contains(_)) }.
        toSeq:_*)

prun_id_htoken.size

timeit.uptonow / 60.0
// 0.20076514550000002 , in minutes

## Save pruned results to file

In [ ]:
write_int_to_int(prun_htoken_collectfreq, PATH_prun_htoken_collectfreq)

In [ ]:
write_int_to_intList(prun_htoken_id, PATH_prun_htoken_id)

In [ ]:
write_int_to_intList(prun_id_htoken, PATH_prun_id_htoken)

## Load maps (pruned)
## # start from here

In [ ]:
// time it
timeit.start

val prun_htoken_collectfreq: MutHashMap[Int, Int] = MutHashMap[Int, Int]()
val prun_id_htoken: MutHashMap[Int, List[Int]] = MutHashMap[Int, List[Int]]()
val prun_htoken_id: MutHashMap[Int, List[Int]] = MutHashMap[Int, List[Int]]()
val id_name: MutHashMap[Int, String] = MutHashMap[Int, String]()
val token_hash: MutHashMap[String, Int] = MutHashMap[String, Int]()
val name_id: MutHashMap[String, Int] = MutHashMap[String, Int]()

In [ ]:
load_mutmap_int_int(PATH_prun_htoken_collectfreq, prun_htoken_collectfreq)
load_mutmap_int_intList(PATH_prun_id_htoken, prun_id_htoken)
load_mutmap_int_intList(PATH_prun_htoken_id, prun_htoken_id)
load_mutmap_int_string(PATH_id_name, id_name)
load_mutmap_string_int(PATH_token_hash, token_hash)
load_mutmap_string_int(PATH_name_id, name_id)

In [ ]:
// time it
timeit.uptonow / 60.0
// 2.1530588158666664 , in minutes

In [ ]:
print_memory()

// Used Memory:  2569
// Free Memory:  1782
// Total Memory: 4352
// Max Memory:   5461

# Queries & Truth

In [ ]:
// requires: having added tinyir to classpath, having added the qrels, i.e. "relevance-judgements.csv" in root 
// builds truth, an object, whose only method .judgements("query-ID") returns the set of all document-IDs deemed 
// relevant to that query, note that these document-IDs are provided as List[String]
// observe that query-ID is a string of an integer between 51 and 90 -> 40 queries in total
import ch.ethz.dal.tinyir.lectures._
val truth = new TipsterGroundTruth(files_path + "/relevance-judgements.csv")

// how to use it, example:
truth.judgements("51")
// observe that the size of relevant documents varies between queries, with the minimum being 52 and the maximum 894
truth.judgements.values.map(x => x.size).min
truth.judgements.values.map(x => x.size).max

In [ ]:
// requires: having added the file "questions-descriptions.txt" to source
// This cell will build a list (can be Stream if required) of query tokens. 
// Note that the 16 is hard-coded to ignore the first 15 characters of these <title> line, which all read 
// "<title> Topic: "
import scala.io.Source
val numPattern = "[0-9]+".r

val title = Source.fromFile(files_path +"questions-descriptions.txt").getLines().filter(_.startsWith("<title>"))
                .map(_.substring(16).trim).map(x => token_filter(x)).toList

val num = Source.fromFile(files_path +"questions-descriptions.txt").getLines().filter(_.startsWith("<num>"))   
                .map(x => numPattern.findFirstIn(x.toString).get.substring(1)).toList

val query = num zip title

## Term-Frequency Model

In [ ]:
// DEFINE AUXILLARY FUNCTIONS

// get inverse-document frequency (idf)
// is defined as the logarithmically scaled inverse fraction of the documents that contain the word, 
// obtained by dividing the total number of documents by the number of documents containing the term, and then 
// taking the logarithm of that quotient.

def hash_query(query: (String, List[String])) = {
    (query._1, query._2.map(x => token_hash.getOrElse(x,-1)).filter(prun_htoken_id.keys.toSet.contains(_)).toSet)
}

val corpus_size = prun_id_htoken.size
def get_idf(query: Set[Int]) = {
    query.map(x => x -> Math.log(corpus_size / prun_htoken_id(x).size)).toMap
}

// get term frequency in a specific document (doc)
def get_tf(query: Set[Int],doc: Int) = {
    prun_id_htoken(doc).filter(query.contains(_)).groupBy(identity).mapValues(_.size)
}

// get tf-idf is defined as tf-idf = tf * idf
def get_tf_idf(query: Set[Int],doc:Int) = {
    get_tf(query,doc).map(x => x._1 -> x._2 * get_idf(query).getOrElse(x._1,0.toDouble))
}

In [ ]:
// Handle a Query --> take in a query, produce a ranking
def handle(query: (String, List[String])) = {
    val hashed_query = hash_query(query)
    val doc_set = hashed_query._2.map(x => prun_htoken_id(x)).flatten.toSet
    val ranking = doc_set.map(x => x -> get_tf_idf(hashed_query._2,x).values.sum).toSeq.sortBy(-_._2)
                    .take(100).map(x => x._1).toList    
    (query._1,ranking)
}

In [ ]:
// test it out.
handle(query(35))

In [ ]:
// Does it work on mass-answering queries?
// It takes nearly 8 minutes though, so about 12 seconds per query on average. 
// Potential speed improvements: Write a function that reduces document collection in the first place.
timeit.start
val answers = query.map(x => handle(x)).toMap.mapValues(_.map(x => id_name(x)))
timeit.uptonow / 60.0

## Evaluation

In [ ]:
// The object Inspector contains all functions required to calculate the evaluation metrics (Precision, Recall, 
// F1-Score and MAP (mean average precision))

object Inspector
{
// calculates average precision for a given answer (returned result of query)
def badass1(retriev2: List[String], relev: Array[String], bounded: Boolean=false): Double ={
    val retriev = retriev2.map(_.replace("-", ""))
    // remember to remove the "-" hyphens from the prediction for comparison purposes
    (retriev.map(relev.contains(_)) // produces a boolean list with true where element belongs to relevant
        .scanLeft(0){case (sum, next) => if(next) sum + 1 else sum}.tail // creates cumulative count of the booleans
        .zipWithIndex.map(x => x._1.toDouble / (x._2 + 1)) // calculates average precision for each element
        .zip(retriev.map(relev.contains(_))) // combines average precision with the boolean list from the start
        .filter(_._2) // to filter out the ones that are not relevant 
        .map(_._1).sum // calculates the numerator (sums up the precision for all elements that are relevant)
        )/ (if (bounded) retriev.size else relev.size) // divides by numerator (depending on bounded or not)
    }

// calculates mean average precision over a set of queries. 
def badass2(retriev_all: Map[String, List[String]], relev_all: Map[String, Array[String]], 
            bounded: Boolean=false): Double = {
    (retriev_all.map(x => Inspector.badass1(x._2,relev_all(x._1),bounded)) // calculate average precision for each query
    .sum)/(retriev_all.size) // calculates mean average precision (average precision over all queries)    
}
// Classic Precision and Recall for a given query, not striclty necessary. 
def evaluate(retriev: List[String], relev: Array[String])={
    val TP = retriev.filter(relev.contains(_)).size.toDouble
    val precision = TP / retriev.size
    val recall = TP / relev.size
    (precision,recall)
}

}

In [ ]:
// Example Usage
val query_ID = "51"
Inspector.badass1(answers(query_ID),truth.judgements(query_ID),bounded=true)
Inspector.badass2(answers,truth.judgements,bounded=true) // answers are my predictions, truth.judgements from tinyIR


In [ ]:
// investigate which queries cause problems.
val query_ID = "67"
truth.judgements(query_ID).size // How many relevant documents exist for that query?

// Give a list of (AP, query_ID) for some easy investigation
answers.map(x => Inspector.badass1(x._2,truth.judgements(x._1),bounded=true)).zip(answers.keys)//.filter(_._1<0.1)//.size

// Look at a particular original query:
query.toMap.get(query_ID)

## Language model score

In [ ]:
answers.keys.toList.zipWith(answers.keys.toList)

In [ ]:
answers.map(x => Inspector.badass1(x._2,truth.judgements(x._1),bounded=true))

In [ ]:
// // collection tf
val collection_size = prun_htoken_collectfreq.foldLeft(0)(_+_._2)

val pruned_token_set = prun_htoken_collectfreq.keys.toSet

// discards log scores
def unfold_name_time(score :(List[(String, Double)], Double)) = {
    (score._1.unzip._1, score._2)
}

// returns id's of docs in which the most query tokens appear in
def reduce_candidate_doc(query: (String, List[Int]), 
                         prun_htoken_id: MutHashMap[Int, List[Int]] = prun_htoken_id, 
                         take_k_results: Int = 100) = {
    
    // doc_id -> # of occurances
    val doc_occurance = query._2.flatMap(token => prun_htoken_id(token)).
        groupBy(identity).mapValues(_.size)
    
    // sorted iterator of # of occurances
    val intersect_value = doc_occurance.values.toSet.toList.sorted.reverse.toIterator
    var iter = intersect_value.next
    
    var cur_doc_occurance = doc_occurance.filter(_._2 > iter - 1)
    
    while((cur_doc_occurance.size < take_k_results) & (intersect_value.hasNext)) {
        iter = intersect_value.next
        cur_doc_occurance = doc_occurance.filter(_._2 > iter - 1)
    }
    cur_doc_occurance.keys.toList
}

val query_hash = query.map{ 
    case (id, str) => (id, str.
                       flatMap(x => token_hash.get(x)).filter(pruned_token_set.contains(_))
                      )}

val lambda = 0.01 // smoothing parameter

In [ ]:
def lang_query(query: (String, List[Int]),
               method: String = "index",
               prun_htoken_collectfreq: MutHashMap[Int, Int] = prun_htoken_collectfreq,
               collection_size: Int = collection_size,
               lambda: Double = lambda, 
               prun_id_htoken: MutHashMap[Int, List[Int]] = prun_id_htoken, 
               prun_htoken_id: MutHashMap[Int, List[Int]] = prun_htoken_id, 
               take_k_results: Int = 100) = {
    
    // time it
    timeit.start
    
    // list of doc id's containing tokens in query
    def candidate_doc(): List[Int] = method match {
        case "index" => reduce_candidate_doc(query = query, take_k_results = take_k_results)
        case "no_index" => (0 to 100000-1).toList
        case "test" => (1 to 2).toList
        case _ => throw new Exception("Please choose either 'index' or 'no_index'")
    }
    
    // number of tokens in doc
    def doc_size(doc_id: Int) = {
        prun_id_htoken(doc_id).size
    }
    
    // map of tokens to frequency in a given doc
    def doc_tf_map(doc_id: Int) = {
        prun_id_htoken(doc_id).
            groupBy(identity).mapValues(_.size)
    }
    
    // list of (relative) frequency of query tokens in a given doc
    def doc_query_tf(doc_id: Int) = {
        query._2.map(token => 
                     doc_tf_map(doc_id).getOrElse(token, 0).toDouble / doc_size(doc_id))
    }
    
    // list of (relative) frequency of query tokens in the collection
    def query_cf() = {
        query._2.map(token => 
                     prun_htoken_collectfreq(token).toDouble / collection_size)
    }
    
    // this only needs to be calculated once per query
    // (wasteful to call function multiple times)
    val cur_query_cf = query_cf()
    
    //
    def smooth_prob(doc_id: Int) = {
        doc_query_tf(doc_id).zip(cur_query_cf).
            map{case (x, y) => (1 - lambda) * x + lambda * y}
    }
    
    // sum log(x) elements of list
    def doc_lang_score(doc_id: Int) = {
        smooth_prob(doc_id).foldLeft(0.0)(_ + Math.log(_))
    }
    
    val log_scores = candidate_doc().map(doc => 
                                         (id_name(doc), doc_lang_score(doc))
                                         ).sortWith(_._2 > _._2)
    
    (log_scores.take(take_k_results), timeit.uptonow / 60.0)
}

In [ ]:
// with raw log scores
val lang_model_rank = query_hash.take(10).map(query => 
                       (query._1, unfold_name_time(
                           lang_query(query, method = "index"))
                       )
                                                 ).toMap

In [ ]:
val lang_model_runtime = lang_model_rank.values.map(x => x._2)
val lang_model_runtime_average = average(lang_model_runtime)

// before reducing candidates
// Iterable[Double] = List(
//   0.3791437873666667,
//   0.5371110624833333,
//   9.883984977816668,
//   1.4138989739999999,
//   2.41249985935
// )

# Testing ground